## Kernel methods

Author: Julian Lißner

For questions and feedback write a mail to: [lissner@mechbau.uni-stuttgart.de](mailto:lissner@mechbau.uni-stuttgart.de)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle
from importlib import reload

sys.path.extend( ['provided_functions', 'incomplete_functions' ])
import kernels
import model_evaluation as evaluate

import result_check as check
import display_sets as display
import sample_sets as sample

## Classification 

### Feature maps
- __NOTE:__ data will be arranged row wise, each sample is one row, each feature is represented in one column.
- feature maps introduce another dimension to the data
- using the feature map, the classification can be conducted via sign considerations
- the data will be classified into two sets $\hat y$ via
$$ \underline{\hat y} = \text{sign}\big( \underline{\Phi}\,(\underline{x})\,\, \underline a + \underline b \big) $$
for each sample $\underline x$
- the following data is classified in two sets depending on their position on the grid
- Introducing the general formulation for the feature map $\underline{\Phi} $
$$ \underline{\Phi}(\underline{x}) = \begin{bmatrix} x_1& x_2 & x_1\cdot x_2 \end{bmatrix} = [ x_1 \quad x_2 \quad \hat{\Phi}(\underline{x}) ]$$
- the data should be split depending on their position relative to the origin
- the feature map is defined as $\hat{\Phi}(\underline x)= x_1 \cdot x_2$ for each sample in $\underline x$
- $a=[0 \quad 0\quad 1]$ and $b=0$ are given.
-----
__Task:__ Classify the data according to the defined feature map and classification above. 

In [ ]:
data = pickle.load( open('data/feature_split.pkl', 'rb') )
x = data[0][0]
phi_hat = lambda x: #TODO 
phi_x = np.hstack( (x, phi_hat(x)[:,None] ) ) 

weights = np.array( [0,0,1] ) #a in the equation
bias = 0  #b in the equation
y_hat = #TODO #compute the activation of y_hat (the sign is considered below) 
x_negative = x[ #TODO ] #only where y_hat(x) is negative
x_positive = x[ #TODO ] #only where y_hat(x) is positive

display.split_sets( x_negative, x_positive, y_hat, data[0][1])
check.Task_1a( x_negative, x, y_hat) 

- this time the samples should be separated by their distance to the origin
- the feature map is defined as $\hat{\underline{\underline{\Phi}}}(\underline x)= ||x||_2$ for each sample in $\underline x$
- $a=[0 \quad 0\quad 1]$ and $b=-0.5$ are given.

-------
__Task:__ Split the data according to the defined feature map and classification above. 

In [ ]:
x = data[1][0]
phi_hat = lambda #TODO
x = np.hstack( ( x, phi_hat(x)[:,None] ))

weights = #TODO
bias = #TODO
y_hat = #TODO
x_one = #TODO
x_two = #TODO

display.split_sets( x_one, x_two, y_hat, data[1][1])

----------------
### Supervised learning, classification
- the given data represents a checkerboard pattern
- for each input (pixel in the plot below), the output is given in the reference solution
- the checkerboard is defined by the outputs having either value +1 or -1

In [ ]:
display.checkerboard()

- goal is to reconstruct said checkerboard using the given samples
- the data samples are the support points used to represent the checkerboard

In [ ]:
#different input parameters for the reference checkerboard can be tested
# these will alter all the results below
reference_checkerboard = sample.checkerboard_reference( resolution=(100,100), n_x=4, n_y=4) 

n_samples = 500
n_noisy = None #number of noisy/wrongly set samples  #None=default 5%
x, y = sample.checkerboard( reference_checkerboard, n_samples, n_noisy)
print( '\ntarget values found in y:', set( y) , '\n')




scaling = np.array( reference_checkerboard.shape)
x_plot = (x*scaling )
colors = np.array( ['_', 'orange', 'green'])
fig, ax = plt.subplots( figsize=(6,6))
ax.imshow( reference_checkerboard.T)
_=ax.scatter( x_plot[:,0], x_plot[:,1], facecolor=colors[y.astype(int)], edgecolor='k')

----
__Task:__ Split the data into two the two sets based on the output value $y$

In [ ]:
negative_set = y<0
x_negative = x[ negative_set]
y_negative = #TODO
n_negative = len( y_negative)

positive_set = #TODO
x_positive = #TODO
y_positive = #TODO 
n_positive = #TODO

## colors should match the background if split correctly
resolution = reference_checkerboard.shape
fig, ax = plt.subplots( figsize=(6,6))
ax.imshow( reference_checkerboard.T)
ax.scatter( x_positive[:,0]*resolution[0], x_positive[:,1]*resolution[1], color='orange', edgecolor='k' )
ax.scatter( x_negative[:,0]*resolution[0], x_negative[:,1]*resolution[1], color='green', edgecolor='k' )
ax.set_title( 'samples of the checkerboard pattern' )

- the classification is conducted with $ \widehat y(x^\prime) = \text{sign}\big(\sum_{i=1}^n a_i\, k( x_i, x^\prime; \gamma) +b\big)$

- with the two sets at hand, the weights and bias of the model can be computed

- The weights are given by: 
$a_i = \underbrace{\frac1{n^+} (y_i +1)}_{a^+} + \underbrace{\frac1{n^-}(y_i -1)}_{a^-},\qquad\quad  y_i \in \{-1,+1\} $
- the bias is given by:
$b =\frac1{(n^-)^2}\underbrace{\sum_{i^-=1}^{n^-} \sum_{j^-=1}^{n^-} k(x^-_i,x^-_j; \gamma)}_{\sum \sum \underline{\underline K}^-} - \frac1{(n^+)^2} \underbrace{\sum_{i^+=1}^{n^+} \sum_{j^+=1}^{n^+} k(x^+_i,x^+_j; \gamma)}_{\sum \sum \underline{\underline K}^+} $

- using the split sets, the computation of the activation can be reformulated as $\widehat y(x^\prime) = \text{sign}\big(\sum_{i^+} a^+\, k( x_i, x^\prime; \gamma) +\sum_{i^-} a^-\, k( x_i, x^\prime; \gamma) +b\big) $<br>
evaluated for all training samples $x_i$ in both sets (`x_positive` and `x_negative`)

-----
__Task:__ Compute the bias and the weight of the model. Write the `classification` function in 'model_evaluation.py'.
The result check will tell you if your model performs well. If it performs too poor for any gamma, the weights and bias might be buggy.

In [ ]:
gamma = 50#TODO #hyperparameter to tune
#if you have too few 'checkers' then gamma is probably too small


weight_positive = #TODO
weight_negative = #TODO
bias_negative = #TODO.... kernels.gaussian( x_negative, x_negative, gamma)
bias_positive = #TODO
bias = bias_negative - bias_positive 

#TODO #write the function in 'model_evaluation.py'
reload( evaluate) 
check.model( lambda x: evaluate.classification( x, x_negative, x_positive, weight_negative, weight_positive, bias, gamma), reference_checkerboard)

- the decision boundaries of the model can be displayed by evaluation every pixel on the grid
--------- 
__Task:__ Evaluate the model for each value to resample the checkerboard grid.  

In [ ]:
scaling = np.array( reference_checkerboard.shape) 
boundaries = np.zeros( reference_checkerboard.shape) 
for i in range( reference_checkerboard.shape[0] ):
    for j in range( reference_checkerboard.shape[1] ):
        x_prime = np.array( [i, j] )/scaling 
        boundaries[i,j] = evaluate.classification( #TODO..., classify=False )


display.sampled_checkerboard( boundaries, x_positive, x_negative, reference_checkerboard)